In [11]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
import eli5
import pandas as pd
import data 
from collections import Counter
import train
plt.style.use('dark_background')
# sns.set(style="white")
NUM_PATIENTS = 12044
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# prep the dataframe
df = data.get_features() # uses several forms
df = data.rename_values(df) # rename features based on their meaning
df = data.preprocess(df) # preprocess the feats

# convert feats to dummy
df = pd.get_dummies(df, dummy_na=True) # treat na as a separate category
df = data.remove_zero_cols(df) # remove any cols that are all zero

outcomes = data.get_outcomes() # 2 outcomes: iai, and iai_intervention
df = pd.merge(df, outcomes, on='id', how='left')


# set up train / test
np.random.seed(42)
df['cv_fold'] = np.random.randint(1, 7, size=df.shape[0]) # 6 is the test set

# feats / outcome keys
feat_names = [k for k in df.keys() if not k in ['id', 'cv_fold'] and not 'IAI' in k]
outcome_def = 'iai_intervention'

100%|██████████| 49/49 [00:02<00:00, 19.22it/s]
48it [00:07,  6.91it/s]


final shape (12044, 433)


In [16]:
'''
X_feats = pd.get_dummies(df_filt[keys], dummy_na=True) # treat na as a separate category
X_feats = remove_zero_cols(X_feats)
    
df = df[feat_names+ [outcome_def]].replace('True', 1).replace('False', 0)
df = df.replace('nan', -1)
df = df.fillna(-1)
df = df.infer_objects()
df = df.astype(float).astype(int)
'''

# remove unusable features
# X_feats_full= X_feats_full[ks]

# get actual features
# X_np = X_feats_full.values

# y_np = outcomes['iai_intervention']

"\nX_feats = pd.get_dummies(df_filt[keys], dummy_na=True) # treat na as a separate category\nX_feats = remove_zero_cols(X_feats)\n    \ndf = df[feat_names+ [outcome_def]].replace('True', 1).replace('False', 0)\ndf = df.replace('nan', -1)\ndf = df.fillna(-1)\ndf = df.infer_objects()\ndf = df.astype(float).astype(int)\n"

In [19]:
df.keys()

Index(['id', 'ageinyrs', 'EDTriageTime_1', 'InjuryDay_1', 'InjTmKnown_1',
       'TriageTmUnkown_1', 'Certification_1', 'LtCostalTender_1',
       'RtCostalTender_1', 'AbnChestAusc_1',
       ...
       'AbdTenderDegree_1_2.0', 'AbdTenderDegree_1_3.0',
       'AbdTenderDegree_1_4.0', 'AbdTenderDegree_1_nan', 'Costal_1_True',
       'AbdTrauma_or_SeatBeltSign_1_False', 'AbdTrauma_or_SeatBeltSign_1_True',
       'iai', 'iai_intervention', 'cv_fold'],
      dtype='object', length=115)

# predict

In [39]:
def get_pecarn_feats(feat_names):
    '''Get dummy variable names
    '''
    pecarn_feats = ['VomitWretch_1', 'RecodedMOI_1', 'GCSScore_1', 'ThoracicTender_1', 'ThoracicTrauma_1', 
              'Cosatsl_1', 'DecrBreathSound_1', 'AbdDistention_1', 'AbdomenPain_1', 'AbdTenderDegree_1',
              'AbdTrauma_1', 'SeatBeltSign_1', 'DistractingPain_1']
    # InjuryMechanism_1, hypotension?, femure fracture
    ks = set()
    for pecarn_feat in pecarn_feats:
        for feat_name in feat_names:
            if pecarn_feat in feat_name:
                ks.add(feat_name)
    ks = np.array(list(ks))
    return ks
pecarn_feats = get_pecarn_feats(feat_names)

In [38]:
for balancing in ['ros']:
    for model_type in tqdm(['gb', 'logistic', 'dt', 'svm', 'rf', 'mlp2']):
        for num_feats in [5, 10, 20, 30, 52]: #111]:
            # feats = feat_names[:num_feats]
            feats = pecarn_feats[:num_feats]
            out_dir = f'results/outcome={outcome_def}'
            os.makedirs(out_dir, exist_ok=True)
            out_name = f'{model_type}_{len(feats)}_{balancing}'
            train.train(df, feat_names=feats, model_type=model_type, balancing=balancing, outcome_def=outcome_def,
                        out_name=f'{out_dir}/{out_name}.pkl')


 83%|████████▎ | 5/6 [31:47<06:44, 404.24s/it]/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)

100%|██████████| 6/6 [36:08<00:00, 361.35s/it]